In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scape
import katdal
import katpoint
import katsdpscripts
import glob
import re
import pandas as pd 

import csv
import operator

INFO: Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt
INFO: Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


In [3]:
from os import system

In [4]:
print "Scape version: %s"%scape.__version__
print "katdal version: %s"%katdal.__version__
print "katpoint version: %s"%katpoint.__version__
print "katsdpscripts version: %s"%katsdpscripts.__version__

Scape version: 0.1.dev616+master.e836eb2
katdal version: 0.9.6.dev664+master.3ff15ef
katpoint version: 0.8.dev355+master.341001f
katsdpscripts version: 0.1.dev2808+master.790eabe


In [5]:
h5_filename ='/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/04/06/1523035800.h5'
%set_env h5_filename = $h5_filename

env: h5_filename=/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/04/06/1523035800.h5


In [6]:
d = katdal.open(h5_filename);
ant_list,ants = [],[]
[ant_list.append(ant.name) for ant in d.ants]
print(ant_list)

DEBUG: Found int_time=7.996616972 in telstate['sdp_l0_int_time']
DEBUG: Found scale_factor_timestamp=1712000000.0 in telstate[u'i0_scale_factor_timestamp']
DEBUG: Found sync_time=1523010668.0 in telstate[u'i0_sync_time']
DEBUG: Found n_chans=4096 in telstate['sdp_l0_n_chans']
DEBUG: Found bandwidth=856000000.0 in telstate['sdp_l0_bandwidth']
DEBUG: Found center_freq=1284000000.0 in telstate['sdp_l0_center_freq']
DEBUG: Added '0237-233' [radec] (and 1 aliases)
DEBUG: Added '0252-712' [radec] (and 1 aliases)
DEBUG: Added '0408-658' [radec] (and 1 aliases)
DEBUG: Added '3C138' [radec] (and 1 aliases)
DEBUG: Added '0537-441' [radec] (and 1 aliases)
DEBUG: Added '0614-349' [radec] (and 1 aliases)
DEBUG: Added '0723-008' [radec] (and 1 aliases)
DEBUG: Added '0733-174' [radec] (and 1 aliases)
DEBUG: Added '0823-500' [radec] (and 1 aliases)
DEBUG: Added '3C237' [radec] (and 1 aliases)
DEBUG: Added '1127-145' [radec] (and 1 aliases)
DEBUG: Added '1148-001' [radec] (and 1 aliases)
DEBUG: Added '

In [7]:
ant_list.remove('m009') # not working - flatline
ant_list.remove('m022') #dead
ant_list.remove('m038') #suspect
ants=','.join(ant_list)
print ants
%set_env ants=$ants

m000,m001,m002,m003,m004,m005,m006,m008,m010,m011,m012,m013,m014,m015,m016,m017,m020,m021,m023,m024,m025,m026,m027,m029,m030,m031,m032,m033,m034,m036,m039,m040,m041,m042,m043,m044,m045,m046,m047,m048,m049,m050,m051,m052,m053,m054,m055,m056,m057,m058,m059,m060,m061,m062,m063
env: ants=m000,m001,m002,m003,m004,m005,m006,m008,m010,m011,m012,m013,m014,m015,m016,m017,m020,m021,m023,m024,m025,m026,m027,m029,m030,m031,m032,m033,m034,m036,m039,m040,m041,m042,m043,m044,m045,m046,m047,m048,m049,m050,m051,m052,m053,m054,m055,m056,m057,m058,m059,m060,m061,m062,m063


In [8]:
ref_ant='m005'
%set_env ref_ant=$ref_ant

env: ref_ant=m005


In [9]:
matplotlib.rcParams['font.size']=5.0

In [10]:
"""
figure(1,figsize=(15,240))
figure(2,figsize=(15,15))
figure(3,figsize=(15,15))
figure(4,figsize=(15,15))
%run /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'H' -r $ref_ant \
 -a $ants $h5_filename
"""

"\nfigure(1,figsize=(15,240))\nfigure(2,figsize=(15,15))\nfigure(3,figsize=(15,15))\nfigure(4,figsize=(15,15))\n%run /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'H' -r $ref_ant  -a $ants $h5_filename\n"

In [11]:
"""
figure(1,figsize=(15,240))
figure(2,figsize=(15,15))
figure(3,figsize=(15,15))
figure(4,figsize=(15,15))
%run -e /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'V' -r $ref_ant \
 -a $ants $h5_filename
"""

"\nfigure(1,figsize=(15,240))\nfigure(2,figsize=(15,15))\nfigure(3,figsize=(15,15))\nfigure(4,figsize=(15,15))\n%run -e /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'V' -r $ref_ant  -a $ants $h5_filename\n"

# Only calculating - but includes m004

In [12]:
res_h = !python /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'H' -r $ref_ant \
  -a $ants -x '1148-001' $h5_filename
print "Done H"

Done H


In [13]:
res_v = !python /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'V' -r $ref_ant \
  -a $ants -x '1148-001' $h5_filename
print "Done V"

Done V


In [14]:
delays_h = {}
for i,r in enumerate(res_h):
    if r.startswith('Antenna'):
        ant = r[8:12]
        delays_h[ant] = {}
        delays_h[ant]['E'] = float(res_h[i+2].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['N'] = float(res_h[i+3].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['U'] = float(res_h[i+4].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['D'] = float(res_h[i+5].split(':')[1].lstrip().split(' ')[0])
delays_h

{'m000': {'D': 5875.797, 'E': -8.261, 'N': -207.288, 'U': 1.203},
 'm001': {'D': 5869.959, 'E': 1.16, 'N': -171.773, 'U': 1.107},
 'm002': {'D': 5871.467, 'E': -32.096, 'N': -224.241, 'U': 1.264},
 'm003': {'D': 5872.778, 'E': -66.496, 'N': -202.283, 'U': 0.914},
 'm004': {'D': 5872.519, 'E': -123.588, 'N': -252.959, 'U': 1.157},
 'm005': {'D': 5877.82, 'E': -102.088, 'N': -283.12, 'U': 1.475},
 'm006': {'D': 5880.204, 'E': -18.232, 'N': -295.427, 'U': 1.781},
 'm008': {'D': 5874.904, 'E': -93.505, 'N': -535.033, 'U': 3.071},
 'm010': {'D': 5881.439, 'E': 88.12, 'N': -511.88, 'U': 3.807},
 'm011': {'D': 5882.337, 'E': 84.015, 'N': -352.079, 'U': 2.751},
 'm012': {'D': 5863.331, 'E': 140.033, 'N': -368.271, 'U': 3.064},
 'm013': {'D': 5863.474, 'E': 236.835, 'N': -393.474, 'U': 3.778},
 'm014': {'D': 5868.655, 'E': 280.708, 'N': -285.803, 'U': 3.193},
 'm015': {'D': 5916.982, 'E': 210.673, 'N': -219.15, 'U': 2.372},
 'm016': {'D': 5815.612, 'E': 288.157, 'N': -185.865, 'U': 2.531},
 'm0

In [15]:
delays_v = {}
for i,r in enumerate(res_v):
    if r.startswith('Antenna'):
        ant = r[8:12]
        delays_v[ant] = {}
        delays_v[ant]['E'] = float(res_v[i+2].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['N'] = float(res_v[i+3].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['U'] = float(res_v[i+4].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['D'] = float(res_v[i+5].split(':')[1].lstrip().split(' ')[0])
delays_v

{'m000': {'D': 5877.025, 'E': -8.248, 'N': -207.292, 'U': 1.22},
 'm001': {'D': 5870.967, 'E': 1.163, 'N': -171.773, 'U': 1.106},
 'm002': {'D': 5872.218, 'E': -32.092, 'N': -224.242, 'U': 1.271},
 'm003': {'D': 5874.411, 'E': -66.507, 'N': -202.279, 'U': 0.897},
 'm004': {'D': 5874.396, 'E': -123.623, 'N': -252.944, 'U': 1.103},
 'm005': {'D': 5878.919, 'E': -102.088, 'N': -283.12, 'U': 1.475},
 'm006': {'D': 5880.995, 'E': -18.23, 'N': -295.428, 'U': 1.786},
 'm008': {'D': 5875.799, 'E': -93.524, 'N': -535.026, 'U': 3.044},
 'm010': {'D': 5882.956, 'E': 88.107, 'N': -511.875, 'U': 3.789},
 'm011': {'D': 5883.223, 'E': 83.998, 'N': -352.074, 'U': 2.727},
 'm012': {'D': 5864.349, 'E': 140.016, 'N': -368.265, 'U': 3.039},
 'm013': {'D': 5864.223, 'E': 236.82, 'N': -393.467, 'U': 3.75},
 'm014': {'D': 5870.255, 'E': 280.691, 'N': -285.796, 'U': 3.165},
 'm015': {'D': 5918.599, 'E': 210.654, 'N': -219.143, 'U': 2.343},
 'm016': {'D': 5816.706, 'E': 288.161, 'N': -185.874, 'U': 2.391},
 'm

# Cross check between polarizations for derived positions

In [16]:
for a in sorted(delays_v):
    print a,
    print("diff E %06.3f") %(delays_h[a]['E'] - delays_v[a]['E']),
    print("diff N %06.3f") %(delays_h[a]['N'] - delays_v[a]['N']),
    print("diff U %06.3f") %(delays_h[a]['U'] - delays_v[a]['U']),
    print("diff D %06.3f") %(delays_h[a]['D'] - delays_v[a]['D'])
    
# expect differences between E,N and U less than 1cm and Delays less than 3m 

m000 diff E -0.013 diff N 00.004 diff U -0.017 diff D -1.228
m001 diff E -0.003 diff N 00.000 diff U 00.001 diff D -1.008
m002 diff E -0.004 diff N 00.001 diff U -0.007 diff D -0.751
m003 diff E 00.011 diff N -0.004 diff U 00.017 diff D -1.633
m004 diff E 00.035 diff N -0.015 diff U 00.054 diff D -1.877
m005 diff E 00.000 diff N 00.000 diff U 00.000 diff D -1.099
m006 diff E -0.002 diff N 00.001 diff U -0.005 diff D -0.791
m008 diff E 00.019 diff N -0.007 diff U 00.027 diff D -0.895
m010 diff E 00.013 diff N -0.005 diff U 00.018 diff D -1.517
m011 diff E 00.017 diff N -0.005 diff U 00.024 diff D -0.886
m012 diff E 00.017 diff N -0.006 diff U 00.025 diff D -1.018
m013 diff E 00.015 diff N -0.007 diff U 00.028 diff D -0.749
m014 diff E 00.017 diff N -0.007 diff U 00.028 diff D -1.600
m015 diff E 00.019 diff N -0.007 diff U 00.029 diff D -1.617
m016 diff E -0.004 diff N 00.009 diff U 00.140 diff D -1.094
m017 diff E 00.037 diff N -0.012 diff U 00.056 diff D -0.757
m020 diff E 00.043 diff 

In [17]:
# write solutions to file
matchRe = re.findall('[0-9]{10,}', h5_filename)
filename='delay_file_'+''.join(matchRe)+'.txt'

with open(filename, 'w') as f:
    for h,v in zip(delays_h,delays_v):
        print >> f, h,v, mean([delays_h[h]['E'],delays_v[v]['E']]) , \
                    mean([delays_h[h]['N'],delays_v[v]['N']]), \
                    mean([delays_h[h]['U'],delays_v[v]['U']]), \
                    delays_h[h]['D'],delays_v[v]['D']
    f.close()

In [18]:
# sort delay file by antenna name and load antennaLOC
sample = open(filename, 'r')
sorter = csv.reader(sample, delimiter=' ')
sort = sorted(sorter, key=operator.itemgetter(1))

with open('ant_loc.csv', 'r') as f:
    tocs = f.read()

In [19]:
cols = ['Ants', 'Delay', 'East', 'North', 'UP', 'TOC']
df = pd.DataFrame(columns=cols)
for line in sort:
    p = re.compile(line[0]+',\s+\d{4}\.\d{1,3}')
    toc = float(p.search(tocs).group()[6:])
    df = df.append(pd.Series(
                             [
                                line[0],
                                line[-2],
                                line[2],
                                line[3],
                                line[4], 
                                toc
                             ], 
                              index=['Ants', 'Delay', 'East', 'North', 'UP', 'TOC']
                            ), ignore_index = True)

In [20]:
deltaTOC, deltaUP = [], []
deltaTOC.append(0)
deltaUP.append(0)
for i in range(1, len(df.TOC[1:])+1):
    deltaTOC.append(df.TOC[i] - df.TOC[i-1])
    deltaUP.append(float(df.UP[i]) - float(df.UP[i-1]))

em  = [' ' for i in range(len(deltaUP))]
df[' '] =  pd.Series(em,  index = df.index)

df['delta UP']  = pd.Series(deltaUP,  index = df.index)
df['delta TOC'] = pd.Series(deltaTOC, index = df.index)

print('\nDelay Solutions and antenna locations\n')
df


Delay Solutions and antenna locations



,Ants,Delay,East,North,UP,TOC,,delta UP,delta TOC
0,m000,5875.797,-8.2545,-207.29,1.2115,1052.205,,0.0000,0.000
1,m001,5869.959,1.1615,-171.773,1.1065,1051.874,,-0.1050,-0.331
2,m002,5871.467,-32.094,-224.2415,1.2675,1052.210,,0.1610,0.336
3,m003,5872.778,-66.5015,-202.281,0.9055,1051.754,,-0.3620,-0.456
4,m004,5872.519,-123.6055,-252.9515,1.13,1051.976,,0.2245,0.222
5,m005,5877.82,-102.088,-283.12,1.475,1052.430,,0.3450,0.454
6,m006,5880.204,-18.231,-295.4275,1.7835,1052.590,,0.3085,0.160
7,m008,5874.904,-93.5145,-535.0295,3.0575,1053.990,,1.2740,1.400
8,m010,5881.439,88.1135,-511.8775,3.798,1054.756,,0.7405,0.766
9,m011,5882.337,84.0065,-352.0765,2.739,1053.521,,-1.0590,-1.235


In [21]:
# send file to ops server to process and push to github

opsServer='ops.kat.ac.za'
rPath='/home/kat/ops_team_sw/utilities/update_delay_models'
cmd = "sshpass -p {} scp -r {} kat@{}:{}".format('kat', filename, opsServer, rPath)
system(cmd)


32512